
<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-11t1vbu4x-xkBIHmOREQnYnYDH1GDfCg">Community</a>
    </p>
</center>

# <center>Tracing and Evaluating Agno Agents</center>

This guide shows you how to create and evaluate agents with Arize to improve performance. We'll go through the following steps:

* Create an agent using the Agno agents SDK

* Trace the agent activity

* Add evaluations on your traces

In [ ]:
!pip install -q agno duckduckgo-search newspaper4k lxml_html_clean openinference-instrumentation-agno openinference-instrumentation-openai openai arize-otel

## Connect to Arize

### Setup Keys

Copy the Arize `API_KEY` and `SPACE_ID` from your Space Settings page (shown below) to the variables in the cell below.

<center><img src="https://storage.googleapis.com/arize-assets/barcelos/Screenshot%202024-11-11%20at%209.28.27%E2%80%AFPM.png" width="700"></center>

In [ ]:
import os
import nest_asyncio
from getpass import getpass

nest_asyncio.apply()

SPACE_ID = globals().get("SPACE_ID") or getpass(
    "🔑 Enter your Arize Space ID: "
)
API_KEY = globals().get("API_KEY") or getpass("🔑 Enter your Arize API Key: ")
OPENAI_API_KEY = globals().get("OPENAI_API_KEY") or getpass(
    "🔑 Enter your OpenAI API key: "
)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
### Setup Tracing
from arize.otel import register
from openinference.instrumentation.openai import OpenAIInstrumentor
from openinference.instrumentation.agno import AgnoInstrumentor

# Setup OpenTelemetry via our convenience function
tracer_provider = register(
    space_id=SPACE_ID,
    api_key=API_KEY,
    project_name="agno-agents-cookbook",
)

# Start instrumentation
AgnoInstrumentor().instrument(tracer_provider=tracer_provider)
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

## Build your Agno Agent

In [ ]:
from textwrap import dedent

from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.newspaper4k import Newspaper4kTools

policy_research_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    tools=[DuckDuckGoTools(), Newspaper4kTools()],
    description=dedent("""\
        You are a senior policy analyst who advises G20 governments on the
        governance of advanced AI systems. Your expertise spans: 🌐

        • Comparative regulatory analysis (EU AI Act, U.S. EO, China’s draft regs)
        • Risk taxonomy & mitigation frameworks (NIST, ISO, OECD)
        • Multi‑stakeholder negotiation and diplomacy
        • Economic impact modelling and labour‑market studies
        • Standards‑setting processes (IEEE, ISO/IEC)
        • Enforcement mechanisms and audit requirements
        • Rights‑based and ethics‑based approaches to AI governance\
    """),
    instructions=dedent("""\
        1. Discovery Phase 🔍
           – Gather at least 12 authoritative sources: legislation, white‑papers,
             peer‑reviewed studies, think‑tank reports, and reputable news.
           – Prioritise the most recent versions / amendments (≤ 12 months).
           – Identify divergent regional approaches and key stakeholders.

        2. Comparative Analysis 📊
           – Map each region’s regulatory scope, risk tiers, and enforcement powers.
           – Cross‑reference impact assessments and economic forecasts.
           – Highlight areas of convergence and friction (e.g., foundation‑model audits).

        3. Recommendation Draft ✍️
           – Craft a concise, actionable brief for policymakers.
           – Include trade‑offs, implementation timelines, and anticipated market effects.
           – Use bullet points and tables where clarity improves.

        4. Validation & Quality Control ✓
           – Verify every cited statute / article for publication date and authenticity.
           – Ensure balanced representation of civil‑society and industry viewpoints.
           – Flag any major uncertainties or data gaps.
    """),
    expected_output=dedent("""\
        # {Short, Punchy Headline on AI Governance Landscape} 🌐

        ## Executive Summary
        {One‑paragraph snapshot of regulatory momentum and stakes}

        | Region | Current Status | Key Provisions | Enforcement Timeline |
        |--------|---------------|----------------|----------------------|
        | EU     | ...           | ...            | ...                 |
        | U.S.   | ...           | ...            | ...                 |
        | ...    | ...           | ...            | ...                 |

        ## Comparative Findings
        - **Risk Classification:** {...}
        - **Testing & Audit Requirements:** {...}
        - **Penalties & Incentives:** {...}

        ## Strategic Implications
        {Market, innovation, and compliance impacts for enterprises}

        ## Policy Recommendations
        1. **Short‑Term (0‑12 mo):** {...}
        2. **Medium‑Term (1‑3 yrs):** {...}
        3. **Long‑Term (>3 yrs):** {...}

        ## Sources
        {Numbered list, each with publication date and 1‑line relevance note}

        ---
        Prepared by AI Policy Analyst · Published: {current_date} · Last Updated: {current_time}
    """),
    markdown=True,
    show_tool_calls=True,
    add_datetime_to_instructions=True,
)

In [ ]:
policy_research_agent.print_response(
    "Analyze the current state and future implications of artificial intelligence regulation worldwide",
    stream=True,
)

## Add Evals in Arize AX

Evals can be added in the Arize AX dashboard directly. Follow along here to see how:

<video width="100%" controls>
  <source src="https://storage.googleapis.com/arize-phoenix-assets/assets/videos/agno-evals.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>